In [1]:
from stexls.stex.compiler import StexObject, Compiler
from stexls.stex.linker import Linker
from stexls.util.workspace import Workspace
from stexls.vscode import *
from stexls.stex import *

In [2]:
from typing import *
from pathlib import Path
from multiprocessing import Pool
from tqdm import tqdm

In [3]:
%cd ~/MathHub/

/home/marian/MathHub


In [4]:
root = Path('/home/marian/MathHub')
outdir = Path('/tmp/out')

ws = Workspace(root)
ws.include = '.*/smglom/.*'

compiler = Compiler(ws, outdir)

linker = Linker(compiler)

example_file = Path('/home/marian/MathHub/smglom/arithmetics/source/pinumber.en.tex')
#example_file = Path('/home/marian/MathHub/MiKoMH/talks/source/sTeX/ex/sTeX-modules-ex.tex')
#example_file = Path('/home/marian/MathHub/MiKoMH/GenICT/source/programming/en/hardware-software-programming.tex')
#example_file = Path('/home/marian/MathHub/smglom/primes/source/balancedprime.en.tex')

In [5]:
index = {}
objects = [
    linker.compile_and_link(file, index)
    for file in tqdm(ws.files)
]
len(objects)

  4%|▍         | 125/2951 [00:23<03:30, 13.41it/s]

AttributeError: 'TokenWithLocation' object has no attribute 'add_child'